# Isochrone interpolation

In [0]:
# ISOCHRONES_PATH = '/datascope/subaru/data/isochrones/dartmouth/import/afep0_cfht_sdss_hsc_nb_bosz/isochrones.h5'
ISOCHRONES_PATH = '/datascope/subaru/data/isochrones/dartmouth/import/afep0_cfht_sdss_hsc/isochrones.h5'

MAG1, MAG2, MAG3 = 'hsc_g', 'hsc_i', 'hsc_i'

In [0]:
%matplotlib inline

In [0]:
import os, sys
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import AutoMinorLocator, MultipleLocator

In [0]:
plt.rc('font', size=7)

In [0]:
os.environ['CUDA_VISIBLE_DEVICES'] = ''

## Load grid from HDF5

In [0]:
from pfs.ga.isochrones import IsoGrid
import pfs.ga.isochrones.tensorlib as tt

In [0]:
iso = IsoGrid()
iso.load(os.path.expandvars(ISOCHRONES_PATH))

In [0]:
iso.axes.keys()

In [0]:
for k in iso.values.keys():
    print(k, 
          tt.cpu(tt.count_nonzero(tt.isinf(iso.values[k]))),
          tt.cpu(tt.count_nonzero(tt.isnan(iso.values[k]))))

## Interpolate isochrones

In [0]:
# Interpolate as a function of [Fe/H], log t and EEP

f, ax = plt.subplots(1, 1, figsize=(4.5, 4.5), dpi=240)

for fe_h in [-2.5, -2.0, -1.5, -1.0, -0.5]:
    M_ini = tt.tensor(np.linspace(0.75, 0.9, 10000))
    log_t = tt.tensor(np.full_like(M_ini, np.log10(12e9)))
    Fe_H = tt.tensor(np.full_like(M_ini, fe_h))
    
    mi_EEP, mi_M_ini, [hsc_g, hsc_i], mask = iso.interp3d(Fe_H, log_t, M_ini, [iso.values['hsc_g'], iso.values['hsc_i']])
    ax.plot(hsc_g - hsc_i, hsc_i, '-r', lw=0.5)

for m_ini in np.arange(0.75, 0.9, 0.01):
    # Fe_H = tt.tensor(np.linspace(-2.5, -0.5, 10000))
    # Fe_H = tt.tensor(-0.5 - 2.0 * (np.logspace(0, 1, 1000) - 1) / 9)
    Fe_H = tt.tensor(-2.5 + 2.0 * (np.logspace(0, 1, 10000) - 1) / 9)
    log_t = tt.tensor(np.full_like(Fe_H, np.log10(12e9)))
    M_ini = tt.tensor(np.full_like(Fe_H, m_ini))
    
    mi_EEP, mi_M_ini, [hsc_g, hsc_i], mask = iso.interp3d(Fe_H, log_t, M_ini, [iso.values['hsc_g'], iso.values['hsc_i']])
    if mask.numpy().sum() > 0:
        try:
            ax.plot(hsc_g - hsc_i, hsc_i, '-b', lw=0.5)
        except:
            pass

# for eep in np.arange(160, 280, 10):
for eep in np.arange(230, 240, 1):
    Fe_H = tt.tensor(np.linspace(-2.5, -0.5, 100), dtype=tt.float64)
    EEP = tt.tensor(np.full_like(Fe_H, eep))
    log_t = tt.tensor(np.full_like(EEP, np.log10(12e9)))

    mi_EEP, mi_M_ini, [hsc_g, hsc_i], mask = iso.interp3d_EEP(Fe_H, log_t, EEP, [iso.values['hsc_g'], iso.values['hsc_i']])
    ax.plot(hsc_g - hsc_i, hsc_i, '-k', lw=0.5)

ax.invert_yaxis()

# Zoom in on a wiggle

# ax.set_xlim(0.5, 1.0)
# ax.set_ylim(-0.5, 0.5)

In [0]:
M_ini = tt.tensor(np.linspace(0.8, 2.8, 10000))
log_t = tt.tensor(np.full_like(M_ini, 9.0))
Fe_H = tt.tensor(np.full_like(M_ini, 0.0))

In [0]:
mi_EEP, mi_M_ini, [hsc_g, hsc_i], mask = iso.interp3d(Fe_H, log_t, M_ini, [iso.values['hsc_g'], iso.values['hsc_i']])

In [0]:
f, ax = plt.subplots(1, 1)

ax.plot(hsc_g - hsc_i, hsc_i, '.')
ax.invert_yaxis()